<a href="https://colab.research.google.com/github/DavidRamosArchilla/Firewall-AI/blob/main/server_firewall_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing dependencies

In [1]:
!pip install -q --upgrade pip
!pip install -q pyflowmeter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 455.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [2]:
!git clone https://github.com/DavidRamosArchilla/Firewall-AI.git

Cloning into 'Firewall-AI'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 214 (delta 98), reused 164 (delta 49), pack-reused 0
Receiving objects: 100% (214/214), 2.59 MiB | 6.26 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [3]:
!pip install -q -r Firewall-AI/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 5.4 MB/s eta 0:00:00
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [4]:
!pip install -q pyngrok
!pip install -q flask-ngrok

In [5]:
!sudo apt install libpcap0.8
!sudo apt install tcpdump

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libpcap0.8
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 145 kB of archives.
After this operation, 366 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcap0.8 amd64 1.10.1-4build1 [145 kB]
Fetched 145 kB in 1s (107 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libpcap0.8:amd64.
(Reading database ... 121654 files and direc

In [6]:
%cd Firewall-AI

/content/Firewall-AI


In [7]:
%%writefile app.py
from flask import Flask, redirect, url_for, request, jsonify, send_from_directory
from flask_ngrok import run_with_ngrok

import requests
import json

from pyflowmeter import sniffer

from prediction import FirewallModel

TYPES_DICT = {
        'tcp.synflood': '/content/Firewall-AI/test_files/pkt.TCP.synflood.spoofed.pcap',
        'udp.null': 'test_files/pkt.UDP.null.pcapng',
        'Real time traffic': 'Real time traffic',
        'TCP.DOMINATE': 'test_files/pkt.TCP.DOMINATE.syn.ecn.cwr.pcapng',
        'slow_http': 'test_files/http_slowloris.pcap',
        'UDP.fragmented': 'test_files/pkt.UDP.fragmented.pcap',
        'hulk': '/content/Firewall-AI/test_files/hulk.pcap',
        'ICMP.largeempt': 'test_files/ICMP.largeempty.pcap'
    }

model = FirewallModel()
traffic_sniffer = None
sniffer_created = False
app = Flask(__name__)
run_with_ngrok(app)

predicted_data = []

# Serve static files from the dist folder
@app.route('/assets/<path:filename>')
def static_files(filename):
    return send_from_directory('./client/dist/assets', filename)

# Handle 404 errors
@app.errorhandler(404)
def not_found(e):
    return redirect(url_for('dashboard'))

@app.route('/dashboard')
def dashboard():
    return send_from_directory('./client/dist', 'index.html')

@app.route('/traffic-analysis')
def traffic_analysis():
    return send_from_directory('./client/dist', 'index.html')

@app.route("/send_traffic", methods=["POST"])
def post_data():
    if request.is_json:
        data = request.get_json()
        # print("Received new data:", data)
        confidences, predcted_classes = model.predict(data["flows"])
        for (flow, confidence, predcted_class) in zip(
            data["flows"], confidences, predcted_classes
        ):
            if predcted_class != "":  # BENIGN
                # flow = data['flows'][i]
                predicted_data.append(
                    {
                        "type": predcted_class,
                        "src_ip": f'{flow["src_ip"]}:{flow["src_port"]}',
                        "dst_ip": f'{flow["dst_ip"]}:{flow["dst_port"]}',
                        "confidence": f"{confidence:.2%}",
                        "timestamp": flow["timestamp"],
                    }
                )

        print(confidences, predcted_classes)
        return jsonify({"message": "Data received successfully"}), 200
    else:
        return jsonify({"error": "Invalid JSON data in the request"}), 400


@app.route("/get_data", methods=["GET"])
def get_data():
    return jsonify(predicted_data)

@app.route('/start_sniffer', methods=['POST'])
def start_sniffer():
    if request.is_json:
        data = request.get_json()
        test_type = data['file']
        test_file = TYPES_DICT[test_type]
        reload_sniffer(test_file)
        return jsonify({"message": "Data received successfully"}), 200
    else:
        return jsonify({"error": "Invalid JSON data in the request"}), 400

def reload_sniffer(test_file):
    print(test_file)
    global traffic_sniffer
    global sniffer_created
    global predicted_data
    if sniffer_created:
        try:
            traffic_sniffer.stop()
            traffic_sniffer.join()
        except:
            pass
    else:
        sniffer_created = True
    tunnel_url = requests.get("http://localhost:4040/api/tunnels").text
    j = json.loads(tunnel_url)
    tunnel_url = j['tunnels'][0]['public_url']
    print(f'App link:\n{tunnel_url}/dashboard')
    predicted_data = []
    if test_file == 'Real time traffic':
        traffic_sniffer = sniffer.create_sniffer(
            input_interface=None,
            server_endpoint=f'{tunnel_url}/send_traffic',
        )
    else:
        traffic_sniffer = sniffer.create_sniffer(
            input_file=test_file,
            server_endpoint=f'{tunnel_url}/send_traffic',
        )
    traffic_sniffer.start()

if __name__ == '__main__':
    app.run()

Overwriting app.py


#Running the server

In [ ]:
# @title Setting kgrok token
from pyngrok import ngrok
import subprocess
import os


ngrok_token = "" # @param {type:"string"}
ngrok.set_auth_token(ngrok_token)

public_url = ngrok.connect(5000, 'http')

os.chdir('/content/Firewall-AI')
server_process = subprocess.Popen(['python', 'app.py'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print('Public URL:', public_url)
print(f'Dashboard app link: {public_url.public_url}/dashboard')

for line in server_process.stdout:
    print(line, end='')

In [11]:
ngrok.disconnect('https://bda4-35-221-226-143.ngrok-free.app/')